In [1]:
import pandas as pd
import numpy as np
from database.db_api import db_api

db = db_api()

Setting database to ip: 127.0.0.1, port 3306


In [12]:
def show_msus(msus):
    for msu in msus:
        print "Id: {msu.msu}, Type: {msu.msu_type.name}".format(msu = msu)
msus = db.get_items('msus')
show_msus(msus)

Connected to database
Id: 10, Type: socket
Id: 11, Type: socket
Id: 12, Type: socket
Id: 13, Type: http
Id: 14, Type: http
Id: 15, Type: http
Id: 16, Type: read
Id: 17, Type: read
Id: 18, Type: read
Id: 19, Type: read
Id: 20, Type: read
Id: 21, Type: read
Id: 22, Type: read
Id: 23, Type: read
Id: 24, Type: read
Id: 25, Type: regex
Id: 26, Type: regex
Id: 27, Type: regex
Id: 28, Type: regex
Id: 29, Type: regex
Id: 30, Type: regex
Id: 31, Type: regex_route
Id: 32, Type: regex_route
Id: 33, Type: regex_route
Id: 34, Type: write
Id: 35, Type: write
Id: 36, Type: write


In [23]:
def get_msu(msu_id):
    return [msu for msu in msus if msu.msu == msu_id][0]

def get_msu_type(msu_id):
    return get_msu(msu_id).msu_type.name

def sample_msu_ids_by_type():
    types = set([msu.msu_type.name for msu in msus])
    ids = [[m.msu for m in msus if m.msu_type.name == t][0] for t in types]
    return ids

def res_round(x, resolution):
    return ((x / resolution).round() * resolution).astype(x.dtype)

def get_msu_df(msu):
    print "Getting dataframe for msu {msu.msu} ({msu.msu_type.name})".format(msu=msu)
    df = db.get_msu_full_df(msu)
    trange = (max(df.TIME) - min(df.TIME)) * 1e-9
    spp = round(trange / len(df), 2)
    print "\n # Points: {}\n Time range: {} seconds\n Points / second: ~{}".format(
        len(df), trange, 1/spp)

    rounded_time = res_round(df.TIME, spp * 1e9)

    epoch = ((rounded_time - min(rounded_time)) / (spp * 1e9)).astype(int)
    df = df.assign(msu_id = msu.msu)
    df = df.assign(epoch = epoch)
    df = df.set_index('TIME')
    df = df.assign(TIME = df.index)
    return df    

AGGREGATE_STAT_TYPES = (
 "ERROR_COUNT",
 "MSU_USER_TIME", "MSU_KERNEL_TIME",
 "MSU_MINOR_FAULTS", "MSU_MAJOR_FAULTS",
 "MSU_VOL_CTX_SW","MSU_INVOL_CTX_SW",
)

def make_numeric(df):
    for x in df.columns:
        df[x] = pd.to_numeric(df[x])

def convert_to_rates(msu, types=AGGREGATE_STAT_TYPES):
    for t in types:
        df[t] = df[t].diff() / df.TIME.diff()

MSU_IDS = sample_msu_ids_by_type()
        
#dataset = pd.concat([get_msu_df(get_msu(i)) for i in MSU_IDS])
dataset = get_msu_df(get_msu(13))
make_numeric(dataset)
convert_to_rates(dataset)


Getting dataframe for msu 13 (http)
Connected to database
Connected to database

 # Points: 4495
 Time range: 2247.80013329 seconds
 Points / second: ~2.0


In [24]:
# sudo pip install tensorflow; sudo pip install keras
from keras.layers import LSTM

dataset

ERROR_COUNT  EXEC_TIME  IDLE_TIME  ITEMS_PROCESSED  \
TIME                                                                      
1512667243227255594          0.0   0.000000   0.000000              0.0   
1512667243727259179          0.0   0.000000   0.000000              0.0   
1512667244227262764          0.0   0.000000   0.000000              0.0   
1512667244727266346          0.0   0.000000   0.000000              0.0   
1512667245227269938          0.0   0.000000   0.000000              0.0   
1512667245727273522          0.0   0.000000   0.000000              0.0   
1512667246227277103          0.0   0.000000   0.000000              0.0   
1512667246727280694          0.0   0.000000   0.000000              0.0   
1512667247227284280          0.0   0.000000   0.000000              0.0   
1512667247727287868          0.0   0.000000   0.000000              0.0   
1512667248227291448          0.0   0.000000   0.000000              0.0   
1512667248727295036          0.0   0.000000   0.000000              0.0   
1512667249227298621          0.0   0.000000   0.000000              0.0   
1512667249727302208          0.0   0.000000   0.000000              0.0   
1512667250227305795          0.0   0.000000   0.000000              0.0   
1512667250727309377          0.0   0.000000   0.000000              0.0   
1512667251227312967          0.0   0.000000   0.000000              0.0   
1512667251727316552          0.0   0.000000   0.000000              0.0   
1512667252227320139          0.0   0.000000   0.000000              0.0   
1512667252727323722          0.0   0.000000   0.000000              0.0   
1512667253231327335          0.0   0.000000   0.000000              0.0   
1512667253731330924          0.0   0.000000   0.000000              0.0   
1512667254231334506          0.0   0.000000   0.000000              0.0   
1512667254731338091          0.0   0.000000   0.000000              0.0   
1512667255231341676          0.0   0.000000   0.000000              0.0   
1512667255731345264          0.0   0.000000   0.000000              0.0   
1512667256231348852          0.0   0.000000   0.000000              0.0   
1512667256731352437          0.0   0.000000   0.000000              0.0   
1512667257231356024          0.0   0.000000   0.000000              0.0   
1512667257731359608          0.0   0.000000   0.000000              0.0   
...                          ...        ...        ...              ...   
1512669476523284812          0.0   0.000013   0.004369        1440868.0   
1512669477023288401          0.0   0.000013   0.004369        1440868.0   
1512669477523291992          0.0   0.000013   0.004369        1440868.0   
1512669478023295577          0.0   0.000013   0.004369        1440868.0   
1512669478523299166          0.0   0.000013   0.004369        1440868.0   
1512669479023302751          0.0   0.000013   0.004369        1440868.0   
1512669479523306342          0.0   0.000013   0.004369        1440868.0   
1512669480023309925          0.0   0.000013   0.004369        1440868.0   
1512669480523313518          0.0   0.000013   0.004369        1440868.0   
1512669481023317104          0.0   0.000013   0.004369        1440868.0   
1512669481523320693          0.0   0.000013   0.004369        1440868.0   
1512669482023324280          0.0   0.000013   0.004369        1440868.0   
1512669482523327860          0.0   0.000013   0.004369        1440868.0   
1512669483023331454          0.0   0.000013   0.004369        1440868.0   
1512669483523335041          0.0   0.000013   0.004369        1440868.0   
1512669484023338626          0.0   0.000013   0.004369        1440868.0   
1512669484523342219          0.0   0.000013   0.004369        1440868.0   
1512669485023345805          0.0   0.000013   0.004369        1440868.0   
1512669485523349395          0.0   0.000013   0.004369        1440868.0   
1512669486023352979          0.0   0.000013   0.004369        1440868.0   
1512669486527356597          0.0   0.000013   

In [25]:
from matplotlib import pyplot
%matplotlib inline

values = dataset.values
# specify columns to plot
groups = [0, 1, 2, 3, 5, 6, 7]
i = 1
# plot each column
pyplot.figure(figsize=(20, 10))
for group in groups:
	pyplot.subplot(len(groups), 1, i)
	pyplot.plot(values[:, group])
	pyplot.title(dataset.columns[group], y=0.5, loc='right')
	i += 1
pyplot.show()